In [1]:
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

C:\Users\sheha\anaconda3\envs\dojo-env\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#citation for the dataset
# 1. Misra, Rishabh and Prahal Arora. "Sarcasm Detection using News Headlines Dataset." AI Open (2023).
# 2. Misra, Rishabh and Jigyasa Grover. "Sculpting Data for ML: The first act of Machine Learning." ISBN 9798585463570 (2021).


In [3]:
data = []
with open("Sarcasm_Headlines_Dataset.json", 'r') as f:
    lines = f.readlines()
    for i in lines:
        print(i)                          # optional - just print it out to explain the next 2 lines
        i.replace("'","")              # this is to replace " ' " with ""
        i.replace("/n","")          # this is to replace "/n" with ""
        obj = json.loads(i)
        data.append(obj)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
sentences =[]
labels =[]
urls =[]

for item in data:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

In [5]:
#split to train and test data
train_size = 20000

train_sentences = sentences[0:train_size]
test_sentences = sentences[train_size:]

train_labels = labels[0:train_size]
test_labels = labels[train_size:]

In [6]:
#instantiate tokenizer
vocab_size =10000
tokenizer = Tokenizer(num_words = vocab_size, oov_token = "<OOV>")
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

#create sequences for train data
max_length = 100
trunc_type='post'
padding_type='post'

train_sequences = tokenizer.texts_to_sequences(train_sentences)
#padding for train data
train_padded = pad_sequences(train_sequences, padding = padding_type,
                             maxlen= max_length, truncating = trunc_type)

#create sequences for test data
test_sequences = tokenizer.texts_to_sequences(test_sentences)
#padding for test data
test_padded = pad_sequences(test_sequences, padding = padding_type,
                             maxlen= max_length, truncating = trunc_type)

print(train_padded[0])
print(train_padded.shape)

[ 328    1  799 3405 2404   47  389 2214    1    6 2614 8863    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
(20000, 100)


In [7]:
# Need this block to get it to work with TensorFlow 2.x
import numpy as np
train_padded = np.array(train_padded)
train_labels = np.array(train_labels)
test_padded = np.array(test_padded)
test_labels = np.array(test_labels)

In [8]:
#instantiate the model
embedding_dim = 16

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') 
])


model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 24)                408       
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [9]:
num_epochs = 30
history = model.fit(train_padded, train_labels, epochs=num_epochs, validation_data=(test_padded, test_labels), verbose=2)

Epoch 1/30
625/625 - 3s - loss: 0.6745 - accuracy: 0.5694 - val_loss: 0.6217 - val_accuracy: 0.6105 - 3s/epoch - 4ms/step
Epoch 2/30
625/625 - 2s - loss: 0.4591 - accuracy: 0.8152 - val_loss: 0.3979 - val_accuracy: 0.8302 - 2s/epoch - 3ms/step
Epoch 3/30
625/625 - 2s - loss: 0.3206 - accuracy: 0.8722 - val_loss: 0.3595 - val_accuracy: 0.8468 - 2s/epoch - 3ms/step
Epoch 4/30
625/625 - 2s - loss: 0.2680 - accuracy: 0.8929 - val_loss: 0.3496 - val_accuracy: 0.8523 - 2s/epoch - 3ms/step
Epoch 5/30
625/625 - 2s - loss: 0.2315 - accuracy: 0.9115 - val_loss: 0.3501 - val_accuracy: 0.8544 - 2s/epoch - 3ms/step
Epoch 6/30
625/625 - 2s - loss: 0.2023 - accuracy: 0.9224 - val_loss: 0.3478 - val_accuracy: 0.8544 - 2s/epoch - 3ms/step
Epoch 7/30
625/625 - 2s - loss: 0.1808 - accuracy: 0.9315 - val_loss: 0.3607 - val_accuracy: 0.8526 - 2s/epoch - 3ms/step
Epoch 8/30
625/625 - 2s - loss: 0.1620 - accuracy: 0.9389 - val_loss: 0.3710 - val_accuracy: 0.8544 - 2s/epoch - 3ms/step
Epoch 9/30
625/625 - 2s 

In [10]:
#test for new sentences
sentence = ["granny starting to fear spiders in the garden might be real", 
            "game of thrones season finale showing this sunday night"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

1/1 [==============================] - 0s 85ms/step
[[9.0806085e-01]
 [5.8077103e-06]]
